In [1]:
import csv
import sys
from datetime import date
import pandas as pd
import selenium.webdriver
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta

In [2]:
#for exception
import linecache
import sys

def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,destin,trDate):
    global none
    # User defined variables for data retreival
    #origin = "BOM" 				# Origin airport code
    #destin = "DEL" 				# Destination airport code
    #trDate = ('30'+'/'+'09'+'/2021')			# Date as 1st command line argument.
    #print(sys.argv)

    """ The following is the Base Url for fetching data from MakeMyTrip Website.
        This URL appears in the search bar after origin, destination and date inputs on the landing page.
        Thus, this URL can be changed based on User Inputs and required data can be fetched.
    """
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng"
    
    try:
        driver = selenium.webdriver.Chrome(r'C:\Users\tusha\Downloads\selenium driver\chromedriver.exe') # Chrome driver is being used.
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  			 # URL requested in browser.
        print ("Webpage found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]' # First box with relevant flight data.

        # Wait until the first box with relevant flight data appears on Screen
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        # Scroll the page till bottom to get full data available in the DOM.
        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...") # No more usage needed.
        driver.quit() 				# Browser Closed.

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
        
        # Extract the required tags 
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')		# Tags with Flight Name
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})				# Tags with Departure City
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})			# Tags with flight stops
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 
    # EOF
    # ----------------------------------------------------------------------------------------------------------
    #print("Records\nFlight Name: "+ str(len(spanFlightName)) + "\nFlightCode: "+ str(len(pFlightCode)) + "\nDept Time: "+ str(len(divDeptTime)) + "\nDept City: "+ str(len(pDeptCity)) + "\nFlight Duration: "+ str(len(pFlightDuration)) + "\nArrival Time: "+ str(len(pArrivalTime)) + "\nArrival City: "+ str(len(pArrivalCity)) + "\nFlight Cost: "+ str(len(spanFlightCost)))
    #print(flightsData)
    #print(outputFile)
    

In [4]:
dates=(pd.date_range(start='26/10/2021', end='26/11/2021')) #dates 

New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   # 1 --->01,2--->02 conversion
    month=str(date.month).zfill(2)
    year=str(date.year)
    
    date=day+'/'+month+'/'+year  #formating date
    New_date.append(date)
print(New_date)    

['26/10/2021', '27/10/2021', '28/10/2021', '29/10/2021', '30/10/2021', '31/10/2021', '01/11/2021', '02/11/2021', '03/11/2021', '04/11/2021', '05/11/2021', '06/11/2021', '07/11/2021', '08/11/2021', '09/11/2021', '10/11/2021', '11/11/2021', '12/11/2021', '13/11/2021', '14/11/2021', '15/11/2021', '16/11/2021', '17/11/2021', '18/11/2021', '19/11/2021', '20/11/2021', '21/11/2021', '22/11/2021', '23/11/2021', '24/11/2021', '25/11/2021', '26/11/2021']


In [5]:
# various list container
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
#transforming scraped data 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    
    df.to_csv('Flight.csv', mode='a', header=False)
        
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [6]:
## applying various state combination and extrating  1 month flight data

list_of_state=['BOM','BLR','DEL','HYD']
# 
for i in range(0,len(list_of_state)):
    orgin=list_of_state[i]
    for_to=list_of_state
    
    
    
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
                
        else: 
            
            print( "-----Skiped-----")
            print(" ")
    
        

--------------
origin = BOM
destin = BOM

-----Skiped-----
 
origin = BOM
destin = BLR

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BOM-BLR-26/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 47
Time Count in Scraping  = 94
City Count in Scraping  = 94
spanFlightCost Count in Scraping  = 47
pFlightStops Count in Scraping  = 47
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,26/10/2021,Mumbai,Bengaluru,11:40,18:55,1 stop via New Delhi,"₹ 5,100"
2,Go First,26/10/2021,Mumbai,Bengaluru,06:05,07:45,Non stop,"₹ 5,102"
3,Go First,26/10/2021,Mumbai,Bengaluru,18:05,19:45,Non stop,"₹ 5,102"
4,Go First,26/10/2021,Mumbai,Bengaluru,22:15,00:05,Non stop,"₹ 5,102"
5,Go First,26/10/2021,Mumbai,Bengaluru,09:50,13:25,1 stop via Goa,"₹ 5,102"
6,Go First,26/10/2021,Mumbai,Bengaluru,09:50,13:25,1 stop via Goa,"₹ 5,102"
7,Go First,26/10/2021,Mumbai,Bengaluru,05:35,13:25,1 stop via Goa,"₹ 5,102"
8,Go First,26/10/2021,Mumbai,Bengaluru,16:00,22:15,1 stop via Ahmedabad,"₹ 5,102"
9,Go First,26/10/2021,Mumbai,Bengaluru,11:50,22:15,1 stop via Ahmedabad,"₹ 5,102"


Lenght :  48
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BOM-BLR-27/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 51
Time Count in Scraping  = 102
City Count in Scraping  = 104
spanFlightCost Count in Scraping  = 51
pFlightStops Count in Scraping  = 51
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,26/10/2021,Mumbai,Bengaluru,11:40,18:55,1 stop via New Delhi,"₹ 5,100"
2,Go First,26/10/2021,Mumbai,Bengaluru,06:05,07:45,Non stop,"₹ 5,102"
3,Go First,26/10/2021,Mumbai,Bengaluru,18:05,19:45,Non stop,"₹ 5,102"
4,Go First,26/10/2021,Mumbai,Bengaluru,22:15,00:05,Non stop,"₹ 5,102"
...,...,...,...,...,...,...,...,...
94,IndiGo,27/10/2021,Mumbai,Bengaluru,15:50,21:05,1 stop via Mangalore,"₹ 8,516"
95,IndiGo,27/10/2021,Mumbai,Bengaluru,10:20,15:25,1 stop via Chennai,"₹ 8,883"
96,IndiGo,27/10/2021,Mumbai,Bengaluru,13:00,17:45,1 stop via Chennai,"₹ 8,883"
97,IndiGo,27/10/2021,Mumbai,Bengaluru,06:45,12:00,1 stop via Chennai,"₹ 8,936"


Lenght :  99
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BOM-BLR-28/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 54
Time Count in Scraping  = 108
City Count in Scraping  = 110
spanFlightCost Count in Scraping  = 54
pFlightStops Count in Scraping  = 54
Transaform Function Running.... 


...

Lenght :  6376
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BLR-HYD-10/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 40
Time Count in Scraping  = 80
City Count in Scraping  = 80
spanFlightCost Count in Scraping  = 40
pFlightStops Count in Scraping  = 40
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,26/10/2021,Mumbai,Bengaluru,11:40,18:55,1 stop via New Delhi,"₹ 5,100"
2,Go First,26/10/2021,Mumbai,Bengaluru,06:05,07:45,Non stop,"₹ 5,102"
3,Go First,26/10/2021,Mumbai,Bengaluru,18:05,19:45,Non stop,"₹ 5,102"
4,Go First,26/10/2021,Mumbai,Bengaluru,22:15,00:05,Non stop,"₹ 5,102"
...,...,...,...,...,...,...,...,...
6411,IndiGo,10/11/2021,Bengaluru,Hyderabad,14:45,19:20,1 stop via Vijayawada,"₹ 5,488"
6412,IndiGo,10/11/2021,Bengaluru,Hyderabad,10:55,19:20,1 stop via Vijayawada,"₹ 5,488"
6413,IndiGo,10/11/2021,Bengaluru,Hyderabad,06:20,13:30,1 stop via Vijayawada,"₹ 5,871"
6414,IndiGo,10/11/2021,Bengaluru,Hyderabad,10:55,17:05,1 stop via Chennai,"₹ 5,871"


Lenght :  6416
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BLR-HYD-11/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 44
Time Count in Scraping  = 88
City Count in Scraping  = 88
spanFlightCost Count in Scraping  = 44
pFlightStops Count in Scraping  = 44
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,26/10/2021,Mumbai,Bengaluru,11:40,18:55,1 stop via New Delhi,"₹ 5,100"
2,Go First,26/10/2021,Mumbai,Bengaluru,06:05,07:45,Non stop,"₹ 5,102"
3,Go First,26/10/2021,Mumbai,Bengaluru,18:05,19:45,Non stop,"₹ 5,102"
4,Go First,26/10/2021,Mumbai,Bengaluru,22:15,00:05,Non stop,"₹ 5,102"
...,...,...,...,...,...,...,...,...
6455,IndiGo,11/11/2021,Bengaluru,Hyderabad,10:55,19:20,1 stop via Vijayawada,"₹ 5,105"
6456,IndiGo,11/11/2021,Bengaluru,Hyderabad,06:20,13:30,1 stop via Vijayawada,"₹ 5,488"
6457,IndiGo,11/11/2021,Bengaluru,Hyderabad,14:45,19:20,1 stop via Vijayawada,"₹ 5,488"
6458,IndiGo,11/11/2021,Bengaluru,Hyderabad,22:55,07:45,1 stop via Chennai,"₹ 5,488"


Lenght :  6460
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=BLR-HYD-12/11/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
EXCEPTION IN (<ipython-input-3-7266f55c7cd2>, LINE 19 "driver.get(baseDataUrl)  			 # URL requested in browser."): Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=94.0.4606.81)

12/11/2021  scraping complete
 
--------------
origin = DEL
destin = BOM

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-26/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
EXCEPTION IN (<ipython-input-3-7266f55c7cd2>, LINE 19 "driver.get(baseDataUrl)  			 # URL requested in browser."): Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=94.0.4606.81)

26/10/2021  scraping complete
 
origin = DEL
destin = BLR

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BLR-26/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=fa